# DSPy Setup

In [2]:
import dspy

In [3]:
openai_api_key = ""

In [4]:
dspy.settings.configure(
    lm=dspy.OpenAI(
        model="gpt-4o-mini",
        api_key=openai_api_key,
        max_tokens=4000,
        temperature=0
    )
)

In [5]:
dspy.settings.lm("say hello")

['Hello! How can I assist you today?']

# Tool Setup

In [6]:
import json
import os

schemas_path = os.path.join("..", "data", "simple-3-collection-schemas.json")
with open(schemas_path, "r") as f:
    schemas = json.load(f)

json_schema = json.loads(schemas[0])
len(json_schema["weaviate_collections"])

3

In [7]:
from typing import List
from pydantic import BaseModel

class Property(BaseModel):
    name: str
    data_type: List[str]
    description: str

class Collection(BaseModel):
    name: str
    properties: List[Property]
    envisioned_use_case_overview: str

collections = [Collection.parse_obj(collection) for collection in json_schema["weaviate_collections"]]
collections

[Collection(name='RestaurantMenu', properties=[Property(name='DishName', data_type=['string'], description='The name of the dish offered in the restaurant menu.'), Property(name='Price', data_type=['number'], description='The price of the dish.'), Property(name='IsVegetarian', data_type=['boolean'], description='Indicates if the dish is vegetarian.')], envisioned_use_case_overview='The RestaurantMenu collection stores details about each dish offered in a restaurant, including their names, prices, and vegetarian status. This information allows for menu management and facilitates the creation of customer-friendly menu displays.'),
 Collection(name='CustomerOrders', properties=[Property(name='CustomerName', data_type=['string'], description='The name of the customer who places the order.'), Property(name='TotalAmount', data_type=['number'], description="The total amount for the customer's order."), Property(name='IsTakeaway', data_type=['boolean'], description='Indicates whether the order

In [21]:
# Need to save this for quick reference

# Get Weaviate Collections
from typing import Any

class search_weaviate_collection():
        def run(search_query: str):
                return f"{search_query}"

In [22]:
from dspy.predict.avatar import Tool

tools = [Tool(
    tool=search_weaviate_collection,
    name="Search Code Documentation",
    desc="The documentation for the DSPy Python package, contains helpful code references and conceptual explanations."
)]

In [23]:
import dspy

class SearchAgentSignature(dspy.Signature):
    """You will be given a question. Your task is to answer the question."""
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
        format=lambda x: x.strip(),
    )
    answer: str = dspy.OutputField(
        prefix="Answer:",
        desc="answer to the question",
    )

In [24]:
from dspy.predict.avatar import Avatar

search_agent = Avatar(
    tools=tools,
    signature=SearchAgentSignature,
    verbose=True
)

In [25]:
search_agent(
    question="How does HNSW help with vector indexing?"
)

Starting the task...
Action 1: Search Code Documentation (How does HNSW help with vector indexing?)
Action 2: Finish (HNSW (Hierarchical Navigable Small World) is an algorithm used for efficient vector indexing and nearest neighbor search. It constructs a multi-layered graph where each layer contains a subset of the data points, allowing for fast navigation and retrieval of nearest neighbors. The hierarchical structure enables logarithmic search time, making it suitable for large datasets. HNSW is particularly effective in high-dimensional spaces, providing a balance between accuracy and speed.)


Prediction(
    answer='HNSW (Hierarchical Navigable Small World) is an algorithm used for efficient vector indexing and nearest neighbor search. It constructs a multi-layered graph where each layer contains a subset of the data points, allowing for fast navigation and retrieval of nearest neighbors. The hierarchical structure enables logarithmic search time, making it suitable for large datasets. HNSW is particularly effective in high-dimensional spaces, providing a balance between accuracy and speed.',
    actions=[ActionOutput(tool_name='Search Code Documentation', tool_input_query='How does HNSW help with vector indexing?', tool_output='How does HNSW help with vector indexing?')]
)